In [1]:
from cvrp_parser import parse_cvrplib_uchoa_2014

In [3]:
# read the file Uchoa_et_al_2014/X-n101-k25.vrp
file_path = 'Uchoa_et_al_2014/X-n101-k25.vrp'
#load the text into a string
with open(file_path, 'r') as file:
    vrp_text = file.read()
# parse the CVRP data
inst  = parse_cvrplib_uchoa_2014(vrp_text)

In [4]:
print(inst.name, inst.n_customers, inst.capacity, inst.edge_weight_type)
print("Depot coords:", inst.coords[0])
print("First customer coords:", inst.coords[1])
print("Distance(0,1):", inst.dist(0,1))

X-n101-k25 100 206 EUC_2D
Depot coords: [365. 689.]
First customer coords: [146. 180.]
Distance(0,1): 554.1137067425782


In [5]:
from constructors import clarke_wright_parallel
from utils import build_distance_matrix, is_feasible
from plot_utils import save_solution, plot_solution_from_file

# Clarke–Wright
cw = clarke_wright_parallel(inst.coords, inst.demand, inst.capacity, inst.edge_weight_type)
print("[CW] routes:", len(cw.routes), "cost:", cw.cost,
      "feasible:", is_feasible(cw.routes, inst.demand, inst.capacity, must_cover_all=True, n_customers=inst.n_customers))

# 1) Save to .sol
sol_path = save_solution(inst, cw.routes, cw.cost)  # -> solutions/X-n101-k25/X-n101-k25.sol

# 2) Plot from .sol
png_path = plot_solution_from_file(inst, sol_path)    # -> solutions/X-n101-k25/X-n101-k25.png
print("Wrote:", sol_path, "and", png_path)

[CW] routes: 28 cost: 28941.984425178503 feasible: True
Wrote: solutions\X-n101-k25\X-n101-k25.sol and solutions\X-n101-k25\X-n101-k25.png


In [ ]:
from local_search import local_search
# Local search
# Use verbose=True for quick logging, or pass your own logger
ls = local_search(
    coords=inst.coords,
    demand=inst.demand,
    Q=inst.capacity,
    routes=[r[:] for r in cw.routes],
    edge_weight_type=inst.edge_weight_type,
    verbose=True,
    time_limit_sec=10.0,   # stop early, keep best
    improvement_eps=1e-4
)

print(f"BEST cost: {ls.cost:.2f}, routes: {len(ls.routes)}")
#write the LS solution
sol_path_ls = save_solution(inst, ls.routes, ls.cost)
png_path_ls = plot_solution_from_file(inst, sol_path_ls)
print("Wrote:", sol_path_ls, "and", png_path_ls)

In [10]:
from ils import run_ils, ILSConfig
cfg = ILSConfig(
    time_limit_sec=30.0,
    ls_time_slice_sec=2.0,
    max_passes_without_gain=2,
    k_nearest=20,
    T_init_factor=0.01,
    T_cooling=0.98,
    seed=0,
    verbose=True,
)
ils_out = run_ils(
    coords=inst.coords,
    demand=inst.demand,
    Q=inst.capacity,
    routes_initial=[r[:] for r in cw.routes],
    edge_weight_type=inst.edge_weight_type,
    round_euclidean=False,
    cfg=cfg,
)

print(f"[ILS] best_cost={ils_out.cost:.2f} routes={len(ils_out.routes)} in {ils_out.elapsed_sec:.1f}s (iters={ils_out.iterations})")

# 3) Check, save, plot
ok = is_feasible(ils_out.routes, inst.demand, inst.capacity, must_cover_all=True, n_customers=inst.n_customers)
print("Feasible:", ok)

sol_path = save_solution(inst, ils_out.routes, ils_out.cost)
png_path = plot_solution_from_file(inst, sol_path)
print("Wrote:", sol_path, "and", png_path)

16:12:54 [INFO] ILS start | routes=28 cost=28823.4662 | time_limit=30.0s | T0=288.2347
16:12:54 [INFO] [ACC] iter=1 | cur=28869.6009 (Δ=+46.1346) T=288.2347
16:12:54 [INFO] [REJ] iter=2 | cand=29315.1582 worse by +445.5573 | T=282.4700
16:12:55 [INFO] [ACC] iter=3 | cur=28869.6009 (Δ=+0.0000) T=276.8206
16:12:55 [INFO] [ACC] iter=4 | cur=28694.4466 (Δ=-175.1542) T=271.2842
16:12:55 [INFO]   -> new BEST | iter=4 best=28694.4466
16:12:55 [INFO] [ACC] iter=5 | cur=28694.4466 (Δ=+0.0000) T=265.8585
16:12:55 [INFO] [ACC] iter=7 | cur=28694.4466 (Δ=+0.0000) T=255.3305
16:12:56 [INFO] [ACC] iter=8 | cur=29021.1803 (Δ=+326.7337) T=250.2239
16:12:56 [INFO] [ACC] iter=9 | cur=29021.4402 (Δ=+0.2599) T=245.2194
16:12:56 [INFO] [ACC] iter=10 | cur=29021.4402 (Δ=+0.0000) T=240.3150
16:12:56 [INFO] [ACC] iter=11 | cur=28681.2191 (Δ=-340.2212) T=235.5087
16:12:56 [INFO]   -> new BEST | iter=11 best=28681.2191
16:12:56 [INFO] [ACC] iter=13 | cur=28705.7968 (Δ=+24.5777) T=226.1826
16:12:57 [INFO] [ACC] 

[ILS] best_cost=28418.14 routes=28 in 30.0s (iters=192)
Feasible: True
Wrote: solutions\X-n101-k25\X-n101-k25.sol and solutions\X-n101-k25\X-n101-k25.png
